# Tutorial 1: Quick example of Local AUto-Regressive Average (LAURA)

In [1]:
import os
import mne
import numpy as np

import sys; sys.path.insert(0, '../')
from laura import compute_laura, pos_from_forward

# Definitions

In [2]:
verbose = 0
spacing = 'ico3'
fs_dir = mne.datasets.fetch_fsaverage(verbose=verbose)
subjects_dir = os.path.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = os.path.join(fs_dir, 'bem', 'fsaverage-trans.fif')
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

# Create Standard Forward Model

In [3]:
# Create our own info object, see e.g.:
montage = mne.channels.make_standard_montage("standard_1020")
sfreq = 1000 
info = mne.create_info(montage.ch_names, sfreq, 
    ch_types=['eeg']*len(montage.ch_names), verbose=0)
info.set_montage("standard_1020")

# Create and save Source Model
src = mne.setup_source_space(subject, spacing=spacing, surface='white',
                                    subjects_dir=subjects_dir, add_dist=False,
                                    n_jobs=-1, verbose=verbose)

# Forward Model
fwd = mne.make_forward_solution(info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0, n_jobs=-1,
                                verbose=verbose)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                    use_cps=True, verbose=verbose)
leadfield = fwd['sol']['data']
pos = pos_from_forward(fwd)

C:\Users\Lukas\AppData\Local\Temp\ipykernel_5624\2535841669.py:15: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(info, trans=trans, src=src,
C:\Users\Lukas\AppData\Local\Temp\ipykernel_5624\2535841669.py:15: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(info, trans=trans, src=src,
C:\Users\Lukas\AppData\Local\Temp\ipykernel_5624\2535841669.py:15: RuntimeWarning: joblib not installed. Cannot run in parallel.
  fwd = mne.make_forward_solution(info, trans=trans, src=src,


# Compute LAURA inverse solution

In [4]:
idx = 100
evoked = mne.EvokedArray(leadfield[:, idx][:, np.newaxis], info)
stc = compute_laura(evoked, fwd)

c:\Users\Lukas\Documents\projects\laura\dev\..\laura\laura.py:132: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


# Plot Results

In [5]:
%matplotlib qt

pp = dict(hemi='both', surface='white')
brain = stc.plot(**pp)
if pos[idx, 0]<0:
    brain.add_foci(pos[idx], hemi='lh', color='green')
else:
    brain.add_foci(pos[idx], hemi='rh', color='green')

Using pyvistaqt 3d backend.

Using control points [0.02111491 0.02642977 0.13539263]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [0.02111491 0.02642977 0.13539263]
Using control points [0.02111491 0.02642977 0.13539263]
